In [2]:
import time
import pickle
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy.stats import levy_stable
import tensorflow as tf

import bayesflow as bf

2024-06-10 10:25:12.172051: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-10 10:25:12.173414: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-06-10 10:25:12.193685: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-10 10:25:12.193705: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-10 10:25:12.194198: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [3]:
# Suppress scientific notation for floats
np.set_printoptions(suppress=True)

print(tf.__version__)

# Get rid of annoying tf warning
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

2.15.1


In [4]:
def simulate_levy_trial(a=1, z=0.5, v=0, t=0.2, alpha=2.0, dt=0.001):
    """
    Simulates the response time for one trial from a Lévy-Flight Process
     * param a:     threshold separation (a>0)
     * param z:     relative Starting Point (0<z<1)
     * param v:     drift
     * param t:     non-decisio-time (t>0)
     * param alpha: stability parameter (0<a<2)
     * param dt:    step size for simulation (dt>0)
    
    The function returns a simulated response time. A negative sign indicates a response at the lower threshold
    """
    
    scale = np.power(dt,(1/alpha)) / np.sqrt(2)
    
    bin_size = np.int_(1/dt)  # The decision path is simulated in bins of 1 second
    start = [a*z]
    cnt = 0

    while(True):
        path = np.array(start + np.cumsum(v*dt + levy_stable.rvs(alpha, 0, scale=scale, size=bin_size)))
        if np.any(path < 0): 
            cnt = cnt + np.min(np.where(path < 0))
            return (-(t + cnt*dt)).astype(np.float32)
        
        if np.any(path > a): 
            cnt = cnt + np.min(np.where(path > a))
            return (t + cnt*dt).astype(np.float32)
        
        start = path[-1]
        cnt = cnt + bin_size

In [5]:
RNG = np.random.default_rng(2024)

In [6]:
def LF_prior():
    """
    Draws one set of marginally informative priors for the 7-parameter Levy-Flight-Model
    v1 nad v2 represent drift rates for two different stimulus types, 
    with positive and negative average slopes of evidence accumulation, respectively.
    
    The function returns a np array with all parameter values
    """
    a_pi   = RNG.gamma(3, 1/3) + 0.1
    a_ni   = RNG.normal(3, 1/3) + 0.1
    v_pi   = RNG.normal(3, 3)
    v_ni   = RNG.normal(3, 3)
    t0     = RNG.gamma(1, 6) + 0.1
    st0    = RNG.beta(1,2) * 2*t0
    alpha  = RNG.beta (6, 2) * 2

    return np.array([a_pi, a_ni, v_pi, v_ni, t0, st0, alpha]).astype(np.float32)

In [7]:
PARAM_NAMES = ['a_pi', 'a_ni', 'v_pi', 'v_ni', 't', 'st', 'alpha']

In [8]:
prior = bf.simulation.Prior(prior_fun=LF_prior, param_names=PARAM_NAMES)

In [9]:
def LF_experiment(theta, n_obs=500):
    """
    Simulates response times for one participant of a lévy-flight experiment.
     * param theta: numpy array with the parameters of the lfm (a,z,v1,v2,t,st,alpha) 
     * param n_obs: (maxiimum) number of observations
    
    The function returns a 2 by n_obs array, where the first column give the stimulus type (0,1)
    and the second column gives teh response times (negative values indicate responese at the lower threshold)
    """
    sim_data = np.zeros([n_obs,2])
    cnd = np.random.randint(0,2,n_obs)
    sim_data[:,0] = cnd
    for i in range(n_obs):
        sim_data[i,1] = simulate_levy_trial(
                            a = theta[0 + cnd[i]], 
                            z = 0.5,
                            v = theta[2 + cnd[i]], # for a 3 condition experiment, this needs 3
                            t = np.random.uniform(theta[4] - theta[5]/2, theta[4] + theta[5]/2), 
                            alpha = theta[6]
                        )   
    return sim_data.astype(np.float32)

In [10]:
simulator = bf.simulation.Simulator(simulator_fun = LF_experiment)
model = bf.simulation.GenerativeModel(prior=prior, simulator=simulator, name="lfm")

INFO:root:Performing 2 pilot runs with the lfm model...
INFO:root:Shape of parameter batch after 2 pilot simulations: (batch_size = 2, 7)
INFO:root:Shape of simulation batch after 2 pilot simulations: (batch_size = 2, 500, 2)
INFO:root:No optional prior non-batchable context provided.
INFO:root:No optional prior batchable context provided.
INFO:root:No optional simulation non-batchable context provided.
INFO:root:No optional simulation batchable context provided.


In [11]:
summary_net = bf.networks.SetTransformer(input_dim=2, 
                                         summary_dim=21,
                                             dense_settings = {'units': 256, 'activation': 'relu'},
                                         num_dense_fc = 3,
                                         name="lfm_summary")

In [12]:
inference_net = bf.networks.InvertibleNetwork(
    num_params=len(prior.param_names),
    coupling_settings={"dense_args": dict(kernel_regularizer=None), "dropout": False},
    num_coupling_layers = 12,
    name="lfm_inference",
)

In [13]:
amortizer = bf.amortizers.AmortizedPosterior(inference_net, summary_net, name="lfm_amortizer")

In [14]:
prior_means, prior_stds = prior.estimate_means_and_stds(n_draws=100000)
prior_means = np.round(prior_means, decimals=1)
prior_stds = np.round(prior_stds, decimals=1)
print(prior_means, prior_stds)

[[1.1 0.5 3.  3.  0.3 0.2 1.5]] [[0.6 0.  3.  3.  0.2 0.2 0.3]]


In [15]:
def configurator(forward_dict, min_trials=570, max_trials=600):
    """Configure the output of the GenerativeModel for a BayesFlow setup."""

    # Prepare placeholder dict
    out_dict = {}

    # Extract simulated response times
    data = forward_dict["sim_data"]
    num_trials = np.random.randint(min_trials, max_trials + 1)
    idx = np.random.choice(range(data.shape[1]), size=num_trials, replace=False)
    data = data[:, idx, :]

    out_dict["summary_conditions"] = data.astype(np.float32)


    # Make inference network aware of varying numbers of trials
    # We create a vector of shape (batch_size, 1) by repeating the sqrt(num_obs)
    vec_num_obs = np.sqrt(num_trials) * np.ones((data.shape[0], 1))
    out_dict["direct_conditions"] = np.sqrt(vec_num_obs).astype(np.float32)

    # Get data generating parameters
    params = forward_dict["prior_draws"].astype(np.float32)

    # Standardize parameters
    out_dict["parameters"] = ((params - prior_means) / prior_stds).astype(np.float32)

    return out_dict

In [16]:
trainer = bf.trainers.Trainer(
    generative_model=model, 
    amortizer=amortizer, 
    configurator=configurator,
    default_lr=0.00005,
    checkpoint_path="checkpoints//posner"
)

INFO:root:Initialized empty loss history.
INFO:root:Initialized networks from scratch.
INFO:root:Performing a consistency check with provided components...
/tmp/ipykernel_1108975/3106789727.py:25: RuntimeWarning: invalid value encountered in divide
  out_dict["parameters"] = ((params - prior_means) / prior_stds).astype(np.float32)
INFO:root:Done.


In [21]:
f = open("training_data_posner.obj","rb")
train_data  = pickle.load(f)
f.close()

In [23]:
history = trainer.train_offline(
    simulations_dict = train_data, 
    epochs = 100, 
    batch_size = 16
)

Training epoch 1:   0%|          | 0/12500 [00:00<?, ?it/s]

/tmp/ipykernel_1108975/3106789727.py:25: RuntimeWarning: invalid value encountered in divide
  out_dict["parameters"] = ((params - prior_means) / prior_stds).astype(np.float32)


KeyboardInterrupt: 

In [ ]:
history = trainer.load_pretrained_network()

In [1]:
f = bf.diagnostics.plot_losses(history)

NameError: name 'bf' is not defined

In [ ]:
# Generate some validation data
validation_sims = configurator(model(batch_size=1000))

# Extract unstandardized prior draws and transform to original scale
prior_samples = validation_sims["parameters"]

print(
    f"Estimation will be performed on data sets with {validation_sims['summary_conditions'].shape[1]} simulated trials."
)

In [ ]:
# Generate 100 posterior draws for each of the 1000 simulated data sets
post_samples = amortizer.sample(validation_sims, n_samples=100)

# Unstandardize posterior draws into original scale
post_samples = post_samples 

In [ ]:
f = bf.diagnostics.plot_sbc_histograms(post_samples, prior_samples, num_bins=10, param_names=PARAM_NAMES)

In [ ]:
f = bf.diagnostics.plot_sbc_ecdf(post_samples, prior_samples, stacked=False, difference=True, param_names=PARAM_NAMES)

In [ ]:
post_samples = amortizer.sample(validation_sims, n_samples=1000)


In [ ]:
f = bf.diagnostics.plot_recovery(
    post_samples, prior_samples, param_names=prior.param_names, point_agg=np.mean, uncertainty_agg=np.std
)

In [ ]:
f = bf.diagnostics.plot_z_score_contraction(post_samples, prior_samples, param_names=prior.param_names)